In [116]:
!ls -ltrh /oak/stanford/groups/horence/Roozbeh/NOMAD/compactors/TSP_SS2_tissue/TS*/*/classified_compactors.tsv

-rw-rw----+ 1 rdehghan oak_horence   86M Nov 10  2022 /oak/stanford/groups/horence/Roozbeh/NOMAD/compactors/TSP_SS2_tissue/TSP5/Eye/classified_compactors.tsv
-rw-rw----+ 1 rdehghan oak_horence  283M Nov 10  2022 /oak/stanford/groups/horence/Roozbeh/NOMAD/compactors/TSP_SS2_tissue/TSP4/Tongue/classified_compactors.tsv
-rw-rw----+ 1 rdehghan oak_horence  356M Nov 10  2022 /oak/stanford/groups/horence/Roozbeh/NOMAD/compactors/TSP_SS2_tissue/TSP2/Trachea/classified_compactors.tsv
-rw-rw----+ 1 rdehghan oak_horence  408M Nov 10  2022 /oak/stanford/groups/horence/Roozbeh/NOMAD/compactors/TSP_SS2_tissue/TSP10/Skin/classified_compactors.tsv
-rw-rw----+ 1 rdehghan oak_horence  337M Nov 11  2022 /oak/stanford/groups/horence/Roozbeh/NOMAD/compactors/TSP_SS2_tissue/TSP1/Blood/classified_compactors.tsv
-rw-rw----+ 1 rdehghan oak_horence  723M Nov 11  2022 /oak/stanford/groups/horence/Roozbeh/NOMAD/compactors/TSP_SS2_tissue/TSP3/Eye/classified_compactors.tsv
-rw-rw----+ 1 rdehghan oak_horence  674M 

In [5]:
ls /oak/stanford/groups/horence/khoa/data/splash_classification/TSP_SS2_tissue-TSP7-Blood-classified_compactors.tsv 

/oak/stanford/groups/horence/khoa/data/splash_classification/TSP_SS2_tissue-TSP7-Blood-classified_compactors.tsv


In [1]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
path = r"/oak/stanford/groups/horence/khoa/data/splash_classification/TSP_SS2_tissue-TSP5-Eye-classified_compactors.tsv"
path = r"/oak/stanford/groups/horence/khoa/data/splash_classification/TSP_SS2_tissue-TSP7-Blood-classified_compactors.tsv"
# path = r"/oak/stanford/groups/horence/Roozbeh/NOMAD_10x/runs/TSP_10x_gap_0_published/TSP5/Eye/classified_anchors_extendor.tsv"
df = pd.read_csv(path, sep ="\t")


/tmp/ipykernel_106715/2092932526.py:4: DtypeWarning: Columns (13,15,16,27,29,36,38,40,42,43,44,47,48,49,55,57,59,66,68,70) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path, sep ="\t")


In [3]:
df = df[df.anchor_event.isin(["3UTR", "Repeat", "Base_pair_change_1", "Centromere", "splicing", "Internal_splicing", "Base_pair_change_6", "Base_pair_change_7"])]

In [4]:
index_cols = [col for col in df.columns if "index" in col.lower()]
df = df[index_cols + ["anchor_event", "compactor_valid", "valid_abundance", "num_compactor_per_anchor", "ham_dist", "lev_dist"]]


In [6]:
df.anchor_event.unique()

array(['Base_pair_change_6', 'Centromere', '3UTR', 'Repeat',
       'Base_pair_change_7', 'splicing', 'Base_pair_change_1',
       'Internal_splicing'], dtype=object)

In [5]:
anchor_index_dict = df.set_index("anchor_index").to_dict()

In [ ]:
def reduce_compactor(data, n_compactor):
    

In [7]:
def keep_top_n(group, n, count_col, seq_col, index_col, new_col = "top_compactors"):
    sorted_values = sorted(zip(group[count_col], group[seq_col], group[index_col]), key = lambda x: x[0], reverse=True)
    count_seq_dict = {}
    return pd.Series({new_col: sorted_values[:n]})

n = 10  
temp_col = "top_compactors"
count_col = "valid_abundance" 
seq_col = "compactor_valid"
index_col = "compactor_index"
df_top = df.groupby('anchor_index').apply(lambda group:\
                                          keep_top_n(group, n, count_col, seq_col, index_col, temp_col),\
                                          include_groups=False).reset_index()



In [8]:
df_top = df_top.explode(temp_col)
df_top[[count_col, seq_col, index_col]] = df_top[temp_col].apply(pd.Series)
df_top = df_top.drop(temp_col, axis = 1).reset_index()

In [9]:
for key in anchor_index_dict.keys():
    if key not in df_top.columns:
        df_top[key] = df_top.anchor_index.map(anchor_index_dict[key]).values

In [10]:
df_top

,index,anchor_index,valid_abundance,compactor_valid,compactor_index,anchor_event,num_compactor_per_anchor,ham_dist,lev_dist
0,0,1,212,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_250,Base_pair_change_1,559,1,1
1,0,1,205,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_522,Base_pair_change_1,559,1,1
2,0,1,203,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_559,Base_pair_change_1,559,1,1
3,0,1,202,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_18,Base_pair_change_1,559,1,1
4,0,1,201,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_315,Base_pair_change_1,559,1,1
...,...,...,...,...,...,...,...,...,...
317878,44560,81081,45,TTGTTGTACAGATTATTTCATCACCCAGGTATTAAGCCTGGTATCC...,81081_2,Repeat,6,18,16
317879,44560,81081,37,TTGTTGTACAGATTATTTCATCACCCAGGTACCAAGCCTAGTACCC...,81081_3,Repeat,6,18,16
317880,44560,81081,33,TTGTTGTACAGATTATTTCATCACCCAGGTATTAAGCCTAGTACCC...,81081_4,Repeat,6,18,16
317881,44560,81081,30,TTGTTGTACAGATTATTTCATCACCCAGGTATTAAGACTTGTATTC...,81081_6,Repeat,6,18,16


In [11]:
df_top.to_csv(r"/oak/stanford/groups/horence/khoa/data/splash_classification/subset_TSP_SS2_tissue-TSP7-Blood-classified_compactors.tsv", index=None)

In [220]:
df_top

,index,anchor_index,valid_abundance,compactor_valid,compactor_index,anchor_event,num_compactor_per_anchor,ham_dist,lev_dist
0,0,6,157,CATGTACTCTGCGTTGATACCTGTCTCTTATACACATCTCCGAGCC...,6_1,Base_pair_change_7,109,7,7
1,0,6,148,CATGTACTCTGCGTTGATACCTGTCTCTTATACACATCTCCGAGCC...,6_2,Base_pair_change_7,109,7,7
2,0,6,148,CATGTACTCTGCGTTGATACCTGTCTCTTATACACATCTCCGAGCC...,6_6,Base_pair_change_7,109,7,7
3,0,6,147,CATGTACTCTGCGTTGATACCTGTCTCTTATACACATCTCCGAGCC...,6_8,Base_pair_change_7,109,7,7
4,0,6,143,CATGTACTCTGCGTTGATACCTGTCTCTTATACACATCTCCGAGCC...,6_4,Base_pair_change_7,109,7,7
...,...,...,...,...,...,...,...,...,...
42793,8358,14235,41,TTGCTGTGCCTACACCTTCGCCACCCACTATTGGACCAGTTGCTGA...,14235_2,splicing,2,29,24
42794,8359,14242,45,TTGGGAGGCTGAGGCAGGCAGATCACTTGAGGCCAGGAGTTCACCA...,14242_1,Centromere,4,34,12
42795,8359,14242,41,TTGGGAGGCTGAGGCAGGCAGATCACTTGAGGCCAGGAGTTCGAGA...,14242_2,Centromere,4,34,12
42796,8359,14242,31,TTGGGAGGCTGAGGCAGGCAGATCACTTGAGCCCAGGAGTTCAAGA...,14242_3,Centromere,4,34,12


In [ ]:
redundancy_remover = 

In [145]:
df_top.anchor_index.map(anchor_index_dict[key])

0         1
0         1
0         1
0         1
0         1
         ..
44560    16
44560    16
44560    16
44560    16
44560    16
Name: anchor_index, Length: 317883, dtype: int64

In [158]:
df_top

,index,anchor_index,valid_abundance,compactor_valid,compactor_index,anchor_event,num_compactor_per_anchor,ham_dist,lev_dist
0,0,1,212,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_99,Base_pair_change_1,559,1,1
1,0,1,205,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_99,Base_pair_change_1,559,1,1
2,0,1,203,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_99,Base_pair_change_1,559,1,1
3,0,1,202,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_99,Base_pair_change_1,559,1,1
4,0,1,201,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...,1_99,Base_pair_change_1,559,1,1
...,...,...,...,...,...,...,...,...,...
317878,44560,81081,45,TTGTTGTACAGATTATTTCATCACCCAGGTATTAAGCCTGGTATCC...,81081_6,Repeat,6,18,16
317879,44560,81081,37,TTGTTGTACAGATTATTTCATCACCCAGGTACCAAGCCTAGTACCC...,81081_6,Repeat,6,18,16
317880,44560,81081,33,TTGTTGTACAGATTATTTCATCACCCAGGTATTAAGCCTAGTACCC...,81081_6,Repeat,6,18,16
317881,44560,81081,30,TTGTTGTACAGATTATTTCATCACCCAGGTATTAAGACTTGTATTC...,81081_6,Repeat,6,18,16


In [75]:
result.explode("Top_N_Values")["Top_N_Values"].apply(pd.Series)

,0,1
0,212,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...
0,205,GTGCCAAGGCCCATGTACTCTGCGTCTGTCTCTTATACACATCTCC...
1,203,AAGGCCCATGTACTCTGCGTTGATACTGTCTCTTATACACATCTCC...
1,201,AAGGCCCATGTACTCTGCGTTGATACTGTCTCTTATACACATCTCC...
2,202,GCCTTGGCACCCGAGAATTCCAACCCTGTCTCTTATACACATCTCC...
...,...,...
44558,125,TTGTTGGACATTTGGGTTGGTTCCAAGTCTTTGCTATTGTGAATAA...
44559,77,TTGTTGGCCAGGCTGGTCTCAAACTCCTGACCTCAGGTGATCCGCC...
44559,60,TTGTTGGCCAGGCTGGTCTCAAACTCCTGACCTCATGATCCACCTG...
44560,66,TTGTTGTACAGATTATTTCATCACCCAGGTATTAAGCCTAGTACTT...


In [40]:
def keep_top_n(series, n):
    return series.nlargest(n)

grouped_df.groupby('anchor_index')

[['...']]

In [26]:
df.to_csv(r"/oak/stanford/groups/horence/khoa/data/splash_classification/subset_TSP_SS2_tissue-TSP5-Eye-classified_compactors.csv", index=None)

In [20]:
classes = df.groupby("anchor_index").agg({"anchor_event": lambda x: list(x)[1]}).anchor_event.to_list()

In [29]:
df_aligned = pd.read_csv(r"/oak/stanford/groups/horence/khoa/data/splash_classification/aligned_subset_TSP_SS2_tissue-TSP5-Eye-classified_compactors.csv")

In [30]:
df_aligned.

,compactor_index,anchor_index,anchor_event,compactor_valid,valid_abundance,num_compactor_per_anchor,ham_dist,lev_dist,aligned_compactor
0,10627_1,10627,3UTR,AAAAAAAGGGAAGGGGAGGAAAAAAAAAATCACATGATATTGGGAA...,98,2,41,2,AAAAAAAGGGAAGGGGAGGAAAAAAAAAATCACATGATATTGGGAA...
1,10627_2,10627,3UTR,AAAAAAAGGGAAGGGGAGGAAAAAAAAATCACATGATATTGGGAAC...,87,2,41,2,AAAAAAAGGGAAGGGGAGGAAAAAAAAA-TCACATGATATTGGGAA...
2,8932_1,8932,3UTR,AAAAAATAACAAAAAAAGGGAAGGGGAGGAAAAAAAAAATCACATG...,144,4,41,2,AAAAAATAACAAAAAAAGGGAAGGGGAGGAAAAAAAAAA-TCACAT...
3,8932_2,8932,3UTR,AAAAAATAACAAAAAAAGGGAAGGGGAGGAAAAAAAAATCACATGA...,126,4,41,2,AAAAAATAACAAAAAAAGGGAAGGGGAGGAAAAAAAAA--TCACAT...
4,8932_3,8932,3UTR,AAAAAATAACAAAAAAAGGGAAGGGGAGGAAAAAAAATCACATGAT...,34,4,41,2,AAAAAATAACAAAAAAAGGGAAGGGGAGGAAAAAAAA---TCACAT...
...,...,...,...,...,...,...,...,...,...
59889,7018_4,7018,3UTR,TTGTTGCCCAGGCTGGAGTGCAGTGGCATGATCTTGGCTCACTGCA...,38,8,7,6,TTGTTGCCCAGGCTGGAGTGCAGTGGCATGATCTTGGCTCACTGCA...
59890,7018_5,7018,3UTR,TTGTTGCCCAGGCTGGAGTGCAGTGGCGCAATCTTGGCTCACTGCA...,30,8,7,6,TTGTTGCCCAGGCTGGAGTGCAGTGGCGCAATCTTGGCTCACTGCA...
59891,7018_6,7018,3UTR,TTGTTGCCCAGGCTGGAGTGCAGTGGCGTGATCTCGGCTCACTGCA...,23,8,7,6,TTGTTGCCCAGGCTGGAGTGCAGTGGCGTGATCTCGGCTCACTGCA...
59892,7018_7,7018,3UTR,TTGTTGCCCAGGCTGGAGTGCAGTGGCGCAATTTCCACTCACTGCA...,20,8,7,6,TTGTTGCCCAGGCTGGAGTGCAGTGGCGCAATTTCCACTCACTGCA...
